In [21]:
#-- rescaling 50x50 surface into 500x500 i.e 1x1px in 50x50 is 10x10px in 500x500
#-- to do snake body and food should not have same coordinate, game over after collision and pause restart button
import pygame, sys, random
import math as m
from pygame.locals import*
pygame.init()
FPS = 10
fpsclock = pygame.time.Clock()
WINWIDTH = 300
WINHEIGHT = 300
SURFACEWIDTH = 15
SURFACEHEIGHT = 15

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [22]:
class Snake:
        def __init__(self,DISPLAYSURF):
            self.size = 3
            self.rectsize = 1  # draw snake on pixel
            self.node = [[0,0],[3,3],[2,3],[1,3]]
            self.tail = self.node[self.size]
            self.color = (0,255,0)
            self.direction = 'down'
            self.surface = DISPLAYSURF
            self.SPEED =  1 # one pixel at a time
            self.RESCALEFACTOR = 20 # 1px in 50x50 surface is 10px in 500x500 window
            temp = self.size
            while temp != 0:
                self.drawrect(self.node[temp][0],self.node[temp][1],self.rectsize,self.rectsize) 
                temp -= 1
                
        def updateS(self):
                previous = [self.node[1][0],self.node[1][1]]
                if self.direction == 'right' :
                      previous[0] += self.SPEED
                elif self.direction == 'left':
                      previous[0] -= self.SPEED
                elif self.direction == 'up':
                      previous[1] -= self.SPEED
                elif self.direction == 'down':
                      previous[1] += self.SPEED 
                self.node.insert(1,previous)
                self.node.pop()
        
        def addtail(self):
                newtail = [self.tail[0],self.tail[1]]
                if self.direction == 'right' :
                      newtail[0] -= 1
                elif self.direction == 'left':
                      newtail[0] += 1
                elif self.direction == 'up':
                      newtail[1] += 1
                elif self.direction == 'down':
                      newtail[1] -= 1
                self.node.insert(self.size+1,newtail)
                self.size += 1
                self.tail = self.node[self.size]
                
        def collidedItself(self):
            for xy in self.node[2:]:
                if self.node[1][0] == xy[0] and self.node[1][1] == xy[1]:
                    return True
            return False
        
        def outOfbound(self):
            if (self.node[1][0]*self.RESCALEFACTOR >=  WINWIDTH) or (self.node[1][0] < 0) or (self.node[1][1]*self.RESCALEFACTOR >= WINHEIGHT) or (self.node[1][1] < 0):
                return True
            return False
                
        def drawSnake(self):
            sizes = 1
            while sizes != self.size + 1:
                self.drawrect(self.node[sizes][0],self.node[sizes][1],self.rectsize,self.rectsize)
                sizes += 1
                
        def drawrect(self,x,y,xsize,ysize):
                pygame.draw.rect(DISPLAYSURF,(0,255,0),(x*self.RESCALEFACTOR,y*self.RESCALEFACTOR,xsize*self.RESCALEFACTOR,ysize*self.RESCALEFACTOR))

In [23]:
class Food:
    def __init__(self,WINWIDTH,WINHEIGHT,MINWIDTH,MINHEIGHT):
        self.maxwidth = WINWIDTH - 5
        self.maxheight = WINHEIGHT - 5
        self.minwidth =  MINWIDTH + 5
        self.minheight = MINHEIGHT + 5
        self.size = 1
        self.RESCALEFACTOR = 20
        self.color = (255,0,0)
        self.currentX =  -1
        self.currentY =  -1
        self.generateXY()
    
    def generateXY(self):
        self.currentX = random.randint(self.minwidth,self.maxwidth)
        self.currentY = random.randint(self.minheight,self.maxheight)
    
    def drawFood(self,surface):
        #print("food location is",self.currentX,self.currentY)
        self.drawrect(surface,self.currentX,self.currentY,self.size,self.size)
    
    def drawrect(self,surface,x,y,xsize,ysize):
        pygame.draw.rect(surface,self.color,(x*self.RESCALEFACTOR,y*self.RESCALEFACTOR,xsize*self.RESCALEFACTOR,ysize*self.RESCALEFACTOR))
        
        
    

In [24]:
WHITE = (255,255,255)

In [25]:
def rungame(modellr):
    countsec = 0
    snake = Snake(DISPLAYSURF)
    food = Food(SURFACEWIDTH,SURFACEHEIGHT,0,0)
    #-- loop to start game after space is pressed
    score = 0
    string = str(score)
    string = "Snake Game Score:" + string
    pygame.display.set_caption(string)
    #-- main game loop
    while True:
        DISPLAYSURF.fill(WHITE)
        
        
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
        
        #collide(snake,food) create this function
        #model prediciton
        obstacle = obstacles(snake)
        angle = getOrthogonalAngle(snake ,food)
        distance = distanceBetweenSnakeAndFood(snake, food)
        parameter = [[obstacle[0],obstacle[1],obstacle[2],obstacle[3],angle,distance,snake.size]]
        pre = modellr.predict(parameter)
        
        if pre == -1:
            snake.direction = "up"
        elif pre == +1:
            snake.direction = "down"
        elif pre == +3:
            snake.direction = "right"
        elif pre == -3:
            snake.direction = "left"
        
        if (snake.node[1][0] == food.currentX) and (snake.node[1][1] == food.currentY):
            #print("food found")
            snake.addtail()
            food.generateXY()
            score += 1
            string = "Snake Game Score:" + str(score)
            pygame.display.set_caption(string)
        
        countsec += 1
        if countsec == 100:
            food.generateXY()
            countsec = 0
        
        food.drawFood(DISPLAYSURF)
        snake.updateS()
        if (snake.collidedItself()):
            #pygame.display.set_caption("Game Over collided")
            #break
        if (snake.outOfbound()):
            pygame.display.set_caption("Game Over out of bound")
            break
        snake.drawSnake()
        pygame.display.update()
        fpsclock.tick(FPS)

In [26]:
def number(direction):
    if direction == 'up':
        return -1
    elif direction == 'down':
        return +1
    elif direction == 'left':
        return -3
    return +3

In [27]:
def getOrthogonalAngle(snake ,food):
    headx, heady = snake.node[1][0], snake.node[1][1]

    foodx,foody = food.currentX, food.currentY

    base = foodx - headx
    perpendicular = foody - heady

    hypotenuse = m.sqrt(base**2 + perpendicular**2)+0.00001

    angle = m.degrees(m.asin(perpendicular/hypotenuse))%90

    if snake.direction == "right":
        if base >= 0 and perpendicular <= 0:   angle = angle + 0
        elif base <= 0 and perpendicular <= 0: angle = angle + 90
        elif base <= 0 and perpendicular >= 0: angle = angle + 90
        else:                                  angle = angle + 0
    elif snake.direction == "up":
        if base >= 0 and perpendicular <= 0:   angle = angle + 0
        elif base <= 0 and perpendicular <= 0: angle = angle + 0
        elif base <= 0 and perpendicular >= 0: angle = angle + 90
        else:                                  angle = angle + 90
    elif snake.direction == "left":
        if base >= 0 and perpendicular <= 0:   angle = angle + 90
        elif base <= 0 and perpendicular <= 0: angle = angle + 0
        elif base <= 0 and perpendicular >= 0: angle = angle + 0
        else:                                  angle = angle + 90
    else:
        if base >= 0 and perpendicular <= 0:   angle = angle + 90
        elif base <= 0 and perpendicular <= 0: angle = angle + 90
        elif base <= 0 and perpendicular >= 0: angle = angle + 0
        else:                                  angle = angle + 0

    return m.radians(angle-90)/(m.pi/2)

In [28]:
def distanceBetweenSnakeAndFood(snake, food):
    headx, heady = snake.node[1][0], snake.node[1][1]

    foodx, foody = food.currentX, food.currentY

    base = abs(foodx - headx)
    perpendicular = abs(foody - heady)

    hypotenuse = m.sqrt(base**2 + perpendicular**2)
    return hypotenuse/15

In [29]:
def obstacles(snake):
    head = [snake.node[1][0], snake.node[1][1]]
    up = 0
    down = 0
    left = 0
    right = 0
    if (head[0]+1)*snake.RESCALEFACTOR >= WINWIDTH:
        right = 1
    if (head[0]-1)*snake.RESCALEFACTOR < 0:
        left = 1
    if (head[1]+1)*snake.RESCALEFACTOR >= WINHEIGHT:
        down = 1
    if (head[1]-1)*snake.RESCALEFACTOR < 0:
        up = 1
    
    for xy in snake.node[2:]:
        if (head[0]) == xy[0] and (head[1]-1) == xy[1]:
            up = 1
        elif (head[0]) == xy[0] and(head[1]+1) == xy[1]:
            down = 1
        elif (head[0]-1) == xy[0] and (head[1]) == xy[1]:
            left = 1
        elif (head[0]+1) == xy[0] and (head[1]) == xy[1]:
            right = 1
            
    return (up,down,left,right)
    

# Data cleaning

In [30]:
import numpy as np
import pandas as pd

In [31]:
data = pd.read_csv("newdata3.csv")

In [32]:
np.where(np.logical_and(np.logical_and(data['right'] == 1, data['left'] == 1),data['decision'] == 3))

(array([  77,  111,  133,  509, 1055, 1381, 1382, 1393, 1394, 1568, 2089,
        2201, 2277, 2367, 2543, 2725, 3021, 3875, 3893, 4479, 4480, 5350,
        5394, 5556, 5777, 6510, 6881, 7262, 7453, 7860, 7949, 7950, 8168,
        8408, 8723, 9128, 9182, 9183]),)

In [33]:
data.drop(data.index[[  77,  111,  133,  509, 1055, 1381, 1382, 1393, 1394, 1568, 2089,
        2201, 2277, 2367, 2543, 2725, 3021, 3875, 3893, 4479, 4480, 5350,
        5394, 5556, 5777, 6510, 6881, 7262, 7453, 7860, 7949, 7950, 8168,
        8408, 8723, 9128, 9182, 9183]], inplace = True)

In [34]:
np.where(np.logical_and(np.logical_and(data['right'] == 1, data['left'] == 1),data['decision'] == -3))

(array([1159, 1755, 2031, 2168, 2249, 2329, 2408, 2608, 2689, 2755, 3263,
        3264, 3862, 5350, 5737, 5825, 5866, 6339, 6340, 6618, 7057, 7258,
        7532, 7974, 8050, 8840, 8943, 9014, 9118]),)

In [35]:
data.drop(data.index[[1159, 1755, 2031, 2168, 2249, 2329, 2408, 2608, 2689, 2755, 3263,
        3264, 3862, 5350, 5737, 5825, 5866, 6339, 6340, 6618, 7057, 7258,
        7532, 7974, 8050, 8840, 8943, 9014, 9118]], inplace = True)

In [36]:
np.where(np.logical_and(np.logical_and(data['down'] == 1, data['up'] == 1),data['decision'] == +1))

(array([ 707,  732, 1686, 2215, 3436, 3616, 3704, 3976, 4333, 4718, 5281,
        5379, 6825, 7054, 7055, 7104, 7105, 7685, 8105, 8768, 8887, 8915,
        8952, 9076]),)

In [37]:
data.drop(data.index[[ 707,  732, 1686, 2215, 3436, 3616, 3704, 3976, 4333, 4718, 5281,
        5379, 6825, 7054, 7055, 7104, 7105, 7685, 8105, 8768, 8887, 8915,
        8952, 9076]], inplace = True)

In [38]:
np.where(np.logical_and(np.logical_and(data['down'] == 1, data['up'] == 1),data['decision'] == -1))

(array([ 636, 1361, 1362, 2027, 2169, 2242, 2330, 2506, 2683, 2743, 2744,
        2932, 3426, 4244, 4427, 6098, 7052, 7053, 7514, 8815, 8846, 8976,
        9079, 9123, 9124]),)

In [39]:
data.drop(data.index[[ 636, 1361, 1362, 2027, 2169, 2242, 2330, 2506, 2683, 2743, 2744,
        2932, 3426, 4244, 4427, 6098, 7052, 7053, 7514, 8815, 8846, 8976,
        9079, 9123, 9124]], inplace = True)

In [40]:
X = data.drop(labels = ["decision"], axis = 1)
y = data["decision"]

In [41]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9105 entries, 0 to 9220
Data columns (total 7 columns):
up          9105 non-null int64
down        9105 non-null int64
left        9105 non-null int64
right       9105 non-null int64
angle       9105 non-null float64
distance    9105 non-null float64
size        9105 non-null int64
dtypes: float64(2), int64(5)
memory usage: 569.1 KB


In [42]:
len(y_test)

1821

# Model creation

In [43]:
from sklearn.model_selection import train_test_split

In [44]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Logistic Regression

In [45]:
from sklearn.linear_model import LogisticRegression
modellr = LogisticRegression(max_iter=1000)
modellr.fit(X_train, y_train)
prediction = modellr.predict(X_test)
count = 0
for test,actual in zip(prediction, y_test):
    if test == actual:
        count += 1
print(count)
len(prediction)
modellr.score(X_test, y_test)

1432


0.786381109280615

# Random Forest

In [46]:
from sklearn.ensemble import RandomForestClassifier
modelrf = RandomForestClassifier(n_estimators=1000)
modelrf.fit(X_train, y_train)
prediction = modelrf.predict(X_test)
count = 0
for test,actual in zip(prediction, y_test):
    if test == actual:
        count += 1
print(count)
len(prediction)
modelrf.score(X_test, y_test)

1521


0.8352553542009885

# XGBOOST CLASSIFIER

In [47]:
from xgboost import XGBClassifier
modelxg = XGBClassifier(n_estimators=300,max_depth=8)
modelxg.fit(X_train.values, y_train.values)
prediction = modelrf.predict(X_test.values)
count = 0
for test,actual in zip(prediction, y_test.values):
    if test == actual:
        count += 1
print(count)
len(prediction)
modelxg.score(X_test.values, y_test.values)

1521


/home/mahadev/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8517298187808896

# Model save

In [48]:
import pickle
filename = 'finalized_modelbetter.sav'
pickle.dump(modelxg, open(filename, 'wb'))

# Play game

In [50]:
import warnings
warnings.filterwarnings('ignore')
DISPLAYSURF = pygame.display.set_mode((WINWIDTH,WINHEIGHT),0,32)
while True:
    rungame(modelxg)

SystemExit: 